In [1]:
! [ -d "numadic-customer-transaction-prediction" ] || wget https://nuws.co/datahack-dataset
![ -d "numadic-customer-transaction-prediction" ] || tar xvf datahack-dataset

In [2]:
!ls

datahack-dataset  numadic-customer-transaction-prediction  sample_data


In [3]:
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd

dir=Path("numadic-customer-transaction-prediction")
train_path=dir/"train.csv"
test_path=dir/"test.csv"

# Exploration

In [4]:
test_df=pd.read_csv(test_path)
df=pd.read_csv(train_path)
df.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,7.567236,0.394340,-3.245596,14.023978,8.530232,7.537606,14.573126,9.333264,-5.696731,15.244013,12.438567,13.290894,17.257883,4.305430,3.019540,10.584400,13.667496,-4.055133,-1.137908,5.532980,5.053874,-7.687740,10.393046,-0.512886,14.774147,11.434250,3.842499,2.187230,5.868899,10.642131,...,24.259300,5.633293,5.362896,11.002170,-2.871906,19.315753,2.963335,-4.151155,4.937124,5.636008,-0.004962,-0.831777,19.817094,-0.677967,20.210677,11.640613,-2.799585,11.882933,-1.014064,2.591444,-2.741666,10.085518,0.719109,8.769088,12.756676,-3.983261,8.970274,-10.335043,15.377174,0.746072,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,1.235070,5.500793,5.970253,0.190059,4.639536,2.247908,0.411711,2.557421,6.712612,7.851370,7.996694,5.876254,8.196564,2.847958,0.526893,3.777245,0.285535,5.922210,1.523714,0.783367,2.615942,7.965198,2.159891,2.587830,4.322325,0.541614,5.179559,3.119978,2.249730,4.278903,...,10.880263,0.217938,1.419612,5.262056,5.457784,5.024182,0.369684,7.798020,3.105986,0.369437,4.424621,5.378008,8.674171,5.966674,7.136427,2.892167,7.513939,2.628895,8.579810,2.798956,5.261243,1.371862,8.963434,4.474924,9.318280,4.725167,3.189759,11.574708,3.944604,0.976348,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,3.970500,-20.731300,-26.095000,13.434600,-6.011100,1.013300,13.076900,0.635100,-33.380200,-10.664200,-12.402500,-5.432200,-10.089000,-5.322500,1.209800,-0.678400,12.720000,-24.243100,-6.166800,2.089600,-4.787200,-34.798400,2.140600,-8.986100,1.508500,9.816900,-16.513600,-8.095100,-1.183400,-6.337100,...,-7.452200,4.852600,0.623100,-6.531700,-19.997700,3.816700,1.851200,-35.969500,-5.250200,4.258800,-14.506000,-22.479300,-11.453300,-22.748700,-2.995300,3.241500,-29.116500,4.952100,-29.273400,-7.856100,-22.037400,5.416500,-26.001100,-4.808200,-18.489700,-22.583300,-3.022300,-47.753600,4.412300,-2.554300,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.26

In [5]:
df.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

## Check for Nan 

In [6]:
for col in df:
    if df[col].isna().sum()!=0:
        print(col)

Hence the dataset contains no Na values


## Check for outliers

In [7]:
vars=df.drop(["ID_code","target"],axis=1)
var_cols=vars.columns # This will be useful moving forward

mx,mn=0,0
for col in vars.columns:
    if vars[col].max()>mx:
        mx=vars[col].max()
    if vars[col].min()<mn:
        mn=vars[col].min()
print(mx,mn) # Hence there dont seem to be any significant outliers

74.0321 -90.2525


There do not appear to be any outliers

## Check for label imbalance

In [8]:
df.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

Hence data is highly imbalanced. As can be seen above there are roughly 10 times more data points with the label being 0 than data points with the label being 1. 

# Preprocessing

### Addressing the label imbalance

If the imbalance is not addressed, metrics used to measure the efficacy of the model may become misleading. For eg. if the model always predicts the class with more occurence it would still have a very high accuracy

A naive solution would be to randomly sample the datapoints with target 0 till we have equal datapoints of both labels and discarding the remaining datapoints. 

This approach discards a large amount of potential information for the model to learn. Hence this approach won't be used.
To fix the imbalance Synthetic Minority Oversampling Technique(SMOTE) will be performed wherein the population of datapoints with label 1(lower number of datapoints) is randomly sampled and duplicated to increase number of datapoints with label 1.  

In [9]:
from imblearn.over_sampling import SMOTE
oversampler=SMOTE()
resampled_X,resampled_y=oversampler.fit_resample(df[var_cols],df['target'])


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [10]:
np.unique(resampled_y,return_counts=True)

(array([0, 1]), array([179902, 179902]))

Now the number of datapoints with labels 1 and 0 are equal

### Scaling

In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [12]:
scaler=MinMaxScaler()
scaler.fit(resampled_X)
scaled_X=scaler.transform(resampled_X)
scaled_Y=resampled_y.copy()

In [13]:
scaled_X.min(),scaled_X.max()

(0.0, 1.0000000000000002)

### Dimensionality Reduction

In [14]:
from sklearn.decomposition import PCA

In [15]:
variance_preserved=1.0 # Reduction only occurs only if variance_preserved <1.0
pca=PCA(n_components=variance_preserved,svd_solver="full")

if variance_preserved<1.0:
    reduced_X=pca.fit_transform(scaled_X)
else:
    reduced_X=scaled_X.copy()
reduced_Y=scaled_Y.copy()

In [16]:
scaled_X.shape, reduced_X.shape

((359804, 200), (359804, 200))

# Splitting Data



In [17]:
from sklearn.model_selection import train_test_split

In [20]:
trainX,testX, trainY,testY = train_test_split(
    reduced_X, reduced_Y, test_size=0.3, stratify=scaled_Y
)
input_size=trainX.shape[1]

# Model

In [21]:
from tensorflow.keras.layers import Dense,Input
import tensorflow as tf

In [34]:
initializer = tf.keras.initializers.HeNormal()
model=tf.keras.Sequential([
    Input(shape=(input_size,)),
    Dense(300,activation="relu",kernel_initializer=initializer),
    Dense(1000,activation="relu",kernel_initializer=initializer),
    Dense(500,activation="relu",kernel_initializer=initializer),
    Dense(40,activation="relu",kernel_initializer=initializer),
    Dense(1,activation="sigmoid")
])
lr=1e-3
opt=tf.keras.optimizers.Adam(learning_rate=lr)
loss="binary_crossentropy"
metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.FalsePositives()]

model.compile(optimizer=opt,loss=loss,metrics=metrics)

In [35]:
epochs=4
batch_size=100
verbose=2
model.fit(x=trainX,y=trainY,epochs=epochs,batch_size=batch_size,validation_data=(testX,testY),verbose=verbose)

Epoch 1/4
2519/2519 - 45s - loss: 0.4076 - binary_accuracy: 0.8139 - false_negatives_3: 23185.0000 - false_positives: 23690.0000 - val_loss: 0.3664 - val_binary_accuracy: 0.8393 - val_false_negatives_3: 7246.0000 - val_false_positives: 10098.0000
Epoch 2/4
2519/2519 - 44s - loss: 0.3687 - binary_accuracy: 0.8388 - false_negatives_3: 20889.0000 - false_positives: 19721.0000 - val_loss: 0.3542 - val_binary_accuracy: 0.8456 - val_false_negatives_3: 5971.0000 - val_false_positives: 10696.0000
Epoch 3/4
2519/2519 - 44s - loss: 0.3551 - binary_accuracy: 0.8456 - false_negatives_3: 19798.0000 - false_positives: 19084.0000 - val_loss: 0.3584 - val_binary_accuracy: 0.8456 - val_false_negatives_3: 6531.0000 - val_false_positives: 10135.0000
Epoch 4/4
2519/2519 - 44s - loss: 0.3406 - binary_accuracy: 0.8519 - false_negatives_3: 19007.0000 - false_positives: 18286.0000 - val_loss: 0.3318 - val_binary_accuracy: 0.8575 - val_false_negatives_3: 7232.0000 - val_false_positives: 8150.0000


In [36]:
preds=model.predict(testX)
tf.math.round(preds)

<tf.Tensor: shape=(107942, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [1.],
       ...,
       [1.],
       [0.],
       [1.]], dtype=float32)>

In [37]:
tf.math.confusion_matrix(testY,tf.math.round(preds))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[45821,  8150],
       [ 7232, 46739]], dtype=int32)>

# Predicting on Test dataset

In [39]:
inputs=test_df[var_cols]
inputs=scaler.transform(inputs) # Scaling with same scaler
output=tf.math.round(model.predict(inputs))
output=np.array(output).squeeze()

In [40]:
test_df["target"]=pd.Series(output)
test_df.head(10)

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,target
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,-2.0248,-4.3554,13.9696,0.3458,7.5408,14.5001,7.7028,-19.0919,15.5806,16.1763,3.7088,18.8064,1.5899,3.0654,6.4509,14.1192,-9.4902,-2.1917,5.7107,3.7864,-1.7981,9.2645,2.0657,12.7753,11.3334,8.1462,-0.0610,3.5331,9.7804,...,5.4113,3.8302,5.7380,-8.6105,22.9530,2.5531,-0.2836,4.3416,5.1855,4.2603,1.6779,29.0849,8.4685,18.1317,12.2818,-0.6912,10.2226,-5.5579,2.2926,-4.5358,10.3903,-15.4937,3.9697,31.3521,-1.1651,9.2874,-23.5705,13.2643,1.6591,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197,0.0
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,-1.3809,-0.3310,14.1129,2.5667,5.4988,14.1853,7.0196,4.6564,29.1609,0.0910,12.1469,3.1389,5.2578,2.4228,16.2064,13.5023,-5.2341,-3.6648,5.7080,2.9965,-10.4720,11.4938,-0.9660,15.3445,10.6361,0.8966,6.7428,2.3421,12.8678,...,5.6723,3.6873,13.0429,-10.6572,15.5134,3.2185,9.0535,7.0535,5.3924,-0.7720,-8.1783,29.9227,-5.6274,10.5018,9.6083,-0.4935,8.1696,-4.3605,5.2110,0.4087,12.0030,-10.3812,5.8496,25.1958,-8.8468,11.8263,-8.7112,15.9072,0.9812,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760,1.0
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,-4.7057,-3.0422,13.6751,3.8183,10.8535,14.2126,9.8837,2.6541,21.2181,20.8163,12.4666,12.3696,4.7473,2.7936,5.2189,13.5670,-15.4246,-0.1655,7.2633,3.4310,-9.1508,9.7320,3.1062,22.3076,11.9593,9.9255,4.0702,4.9934,8.0667,...,5.5228,3.3159,4.3324,-0.5382,13.3009,3.1243,-4.1731,1.2330,6.1513,-0.0391,1.4950,16.8874,-2.9787,27.4035,15.8819,-10.9660,15.6415,-9.4056,4.4611,-3.0835,8.5549,-2.8517,13.4770,24.4721,-3.4824,4.9178,-2.0720,11.5390,1.1821,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794,0.0
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,0.0095,-5.0659,14.0526,13.5010,8.7660,14.7352,10.0383,-15.3508,2.1273,21.4797,14.5372,12.5527,2.9707,4.2398,13.7796,14.1408,1.0061,-1.3479,5.2570,6.5911,6.2161,9.5540,2.3628,10.2124,10.8047,-2.5588,6.0720,3.2613,16.5632,...,5.3882,3.6775,7.4753,-11.0780,24.8712,2.6415,2.2673,7.2788,5.6406,7.2048,3.4504,2.4130,11.1674,14.5499,10.6151,-5.7922,13.9407,7.1078,1.1019,9.4590,9.8243,5.9917,5.1634,8.1154,3.6638,3.3102,-19.7819,13.4499,1.3104,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108,0.0
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,5.1025,-3.2827,14.1013,8.9672,4.7276,14.5811,11.8615,3.1480,18.0126,13.8006,1.6026,16.3059,6.7954,3.6015,13.6569,13.8807,8.6228,-2.2654,5.2255,7.0165,-15.6961,10.6239,-4.7674,17.5447,11.8668,3.0154,4.2546,6.7601,5.9613,...,5.6688,6.1159,13.2433,-11.9785,26.2040,3.2348,-5.5775,5.7036,6.1717,-1.6039,-2.4866,17.2728,2.3640,14.0037,12.9165,-12.0311,10.1161,-8.7562,6.0889,-1.3620,10.3559,-7.4915,9.4588,3.9829,5.8580,8.3635,-24.8254,11.4928,1.6321,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846,0.0
5,test_5,5.9862,-2.2913,8.6058,7.0685,14.2465,-8.6761,4.2467,14.7632,1.8790,7.2842,-4.9194,-9.1869,14.0581,11.4403,4.2950,13.8808,10.9588,-7.0124,15.1829,18.2710,9.7086,22.9103,6.5080,2.3607,1.7899,13.9555,-7.2154,-2.4084,5.1616,6.9291,0.1141,9.7667,1.2339,17.7117,12.3433,-0.1787,-1.2858,5.5220,13.0498,...,5.6731,5.2368,5.2973,-13.2809,26.6861,3.0711,-5.8410,3.2268,5.

In [41]:
output_df=test_df[["ID_code","target"]]
output_df.to_csv("submission.csv")